In [1]:
# imports
import numpy as np
import os, sys, gc
from scipy.spatial.distance import mahalanobis
import matplotlib.pyplot as plt
%matplotlib inline

from keras.models import Model, load_model 
from keras import backend as K

# mfilieu
util_path = 'C:/ASM/Dropbox/Developments/Jupyter/myutils' if 'C:' in os.getcwd() else '/if3/mm5gg/ASM/Jupyter/myutils'
sys.path.append(util_path)
import my_file_utils as mfileu

Using TensorFlow backend.


In [2]:
#paramteres
hand = 0
num_layer = 3
root_path = 'C:/ASM/DevData/handwash' if 'C:' in os.getcwd() else '/if3/mm5gg/ASM/DevData/handwash'

In [3]:
#load data
fso = mfileu.read_file('DevData/handwash/features', 'features_our.pkl')
fsw = mfileu.read_file('DevData/handwash/features', 'features_wisdm.pkl')
splits = mfileu.read_file('DevData/handwash/features', 'split_indices.pkl')

Xall = fso[hand][:, 5:38]
yall = fso[hand][:, 3]
yall[yall>0] = 1
print("All: ", np.sum(yall==0), np.sum(yall==1))


train_indices = splits[hand]['train']
Xtrain = fso[hand][train_indices, 5:38]
ytrain = fso[hand][train_indices, 3]
ytrain[ytrain>0] = 1
print("Train: ", np.sum(ytrain==0), np.sum(ytrain==1))

val_indices = splits[hand]['val']
Xval = fso[hand][val_indices, 5:38]
yval = fso[hand][val_indices, 3]
yval[yval>0] = 1
print("Val: ",np.sum(yval==0), np.sum(yval==1))

test_indices = splits[hand]['test']
Xtest = fso[hand][test_indices, 5:38]
ytest = fso[hand][test_indices, 3]
ytest[ytest>0] = 1
print("Test: ", np.sum(ytest==0), np.sum(ytest==1), np.sum(ytest==2))

xo = fso[hand][:, 5:38]
xw = fsw[:, 3:]

mu = np.mean(Xtrain, axis=0)
sigma = np.std(Xtrain, axis=0)

Xall = (Xall-mu)/sigma
Xtrain = (Xtrain-mu)/sigma
Xval = (Xval-mu)/sigma
Xtest = (Xtest-mu)/sigma
xo = (xo-mu)/sigma
xw = (xw-mu)/sigma

folder_path = 'DevData/handwash/general_mlp/results'
po = mfileu.read_file(folder_path, 'proba_our_{}.pkl'.format(hand))[:, num_layer-1]
pw = mfileu.read_file(folder_path, 'proba_wisdm_{}.pkl'.format(hand))[:, num_layer-1]

pred_all = (po>=0.5).astype(int)
pred_train = (po[train_indices]>=0.5).astype(int)
pred_val = (po[val_indices]>=0.5).astype(int)
pred_test = (po[test_indices]>=0.5).astype(int)
pred_xw = (pw>=0.5).astype(int)
print(pred_all.shape, pred_train.shape, pred_val.shape, pred_test.shape, pred_xw.shape)

All:  18780 15558
Train:  13521 11202
Val:  1503 1244
Test:  3756 3112 0
(34338,) (24723,) (2747,) (6868,) (377629,)


In [4]:
#keras layers
model_folder = root_path + '/general_mlp/models/hand_{}'.format(hand)    
model_path = model_folder+'/layer_{}.h5'.format(num_layer)
model = load_model(model_path)
model.summary()

models = []
for i in range(1, num_layer+1):
    m = Model(inputs=model.input, outputs=model.get_layer('dense'+str(i)).output)
    models.append(m)
    print('------------ Layer : ', i)
    m.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense1 (Dense)               (None, 64)                2176      
_________________________________________________________________
dropout1 (Dropout)           (None, 64)                0         
_________________________________________________________________
dense2 (Dense)               (None, 64)                4160      
_________________________________________________________________
dropout2 (Dropout)           (None, 64)                0         
_________________________________________________________________
dense3 (Dense)               (None, 64)                4160      
_________________________________________________________________
dropout3 (Dropout)           (None, 64)                0         
_________________________________________________________________
output (Dense)               (None, 1)                 65        
Total para

In [5]:
#layer outputs
all_outs, train_outs, val_outs, test_outs, wisdm_outs = [], [], [], [], []
for i in range(num_layer):
    print("layer: ", i)
    ao = models[i].predict(Xall)
    tro = models[i].predict(Xtrain)
    vo = models[i].predict(Xval)
    to = models[i].predict(Xtest)
    wo = models[i].predict(xw)
    
    all_outs.append(ao)
    train_outs.append(tro)
    val_outs.append(vo)
    test_outs.append(to)
    wisdm_outs.append(wo)

layer:  0
layer:  1
layer:  2


In [6]:
#mean, covariance
mus, icovs = [], []
cond = (yval==1) & (pred_val==1)
for i in range(num_layer):
    vo = val_outs[i][cond]
    mu = np.mean(vo, axis=0)
    cov = np.cov(vo, rowvar=False)
    icov = np.linalg.inv(cov)
    mus.append(mu)
    icovs.append(icov)
    print(icov.shape)

(64, 64)
(64, 64)
(64, 64)


In [7]:
#distance functions
def get_distance_maha(v, mu, icov):
    print(v.shape, mu.shape, icov.shape)
    count = v.shape[0]
    res = np.zeros((count, ))
    for i in range(count):
        res[i] = mahalanobis(v[i], mu, icov)
    
    return res

def get_distance_euclid(v, mu):
    d = v-mu
    d = np.sqrt(np.sum(d**2, axis=1))        
    return d

def get_distance(v, mu, covinv):
    res = np.zeros((v.shape[0], 2))
    res[:, 0] = get_distance_maha(v, mu, covinv)
    res[:, 1] = get_distance_euclid(v, mu)
    return res    

In [8]:
#distances
all_dists, train_dists, val_dists, test_dists, wisdm_dists = [], [], [], [], []
for i in range(num_layer):    
    print('Calculating distance for layer... ', i)
    ad = get_distance(all_outs[i], mus[i], icovs[i])
    trd = get_distance(train_outs[i], mus[i], icovs[i])    
    vd = get_distance(val_outs[i], mus[i], icovs[i])
    td = get_distance(test_outs[i], mus[i], icovs[i])
    wd = get_distance(wisdm_outs[i], mus[i], icovs[i])
    
    all_dists.append(ad)
    train_dists.append(trd)
    val_dists.append(vd)
    test_dists.append(td)
    wisdm_dists.append(wd)

Calculating distance for layer...  0
(34338, 64) (64,) (64, 64)
(24723, 64) (64,) (64, 64)
(2747, 64) (64,) (64, 64)
(6868, 64) (64,) (64, 64)
(377629, 64) (64,) (64, 64)
Calculating distance for layer...  1
(34338, 64) (64,) (64, 64)
(24723, 64) (64,) (64, 64)
(2747, 64) (64,) (64, 64)
(6868, 64) (64,) (64, 64)
(377629, 64) (64,) (64, 64)
Calculating distance for layer...  2
(34338, 64) (64,) (64, 64)
(24723, 64) (64,) (64, 64)
(2747, 64) (64,) (64, 64)
(6868, 64) (64,) (64, 64)
(377629, 64) (64,) (64, 64)


In [9]:
res = {
    'hand':hand,
    'num_layer':num_layer,
    
    'all_outs':all_outs,
    'train_outs':train_outs,
    'val_outs':val_outs,
    'test_outs':test_outs,
    'wisdm_outs':wisdm_outs,
    
    'mus':mus,
    'icovs':icovs,
    
    'all_dists':all_dists,
    'train_dists':train_dists,
    'val_dists':val_dists,
    'test_dists':test_dists,
    'wisdm_dists':wisdm_dists
}

In [10]:
mfileu.write_file('DevData/handwash/general_mlp/ood', 'layer_{}_hand_{}.pkl'.format(num_layer, hand), res)